In [83]:
alignments = ["AGCTACGTGTCGCTGAATCTATGACT", 
              "-GCTA-GAGCA-AGGCAACTGCATCT", 
              "A-CTG-CACCC-ATGAACCTCGCGCT",
              "A-CTG-CACCC-ATGAACCTCTCGCT",
              "A-CTG-CACCC-ATGAACCTCTCGCT",
              "A-CTG-CACCC-ATGAACCTCTCACT",
              "A-CTG-CACCC-ATGAACCTCTCACT"
             ]

def generate_pairs(alignments):
    pairs = []
    for i in range(len(alignments[0])):
        column = [seq[i] for seq in alignments if seq[i] != '-']
        for j in range(len(column)):
            for k in range(j, len(column)):
                pairs.append((column[j], column[k]))
    return pairs

def count_pairs(alignments):
    pair_counts = {}
    for i in range(len(alignments[0])):
        column = [seq[i] for seq in alignments if seq[i] != '-']
        for j in range(len(column)):
            for k in range(j + 1, len(column)):
                pair = tuple(sorted((column[j], column[k])))
                if '-' not in pair:
                    if pair in pair_counts:
                        pair_counts[pair] += 1
                    else:
                        pair_counts[pair] = 1
    return pair_counts

def calculate_frequencies(alignments):
    counts = {'A': 0, 'G': 0, 'C': 0, 'T': 0}
    total = 0
    for alignment in alignments:
        for nucleotide in alignment:
            if nucleotide != '-':
                counts[nucleotide] += 1
                total += 1
    return {nuc: count/total for nuc, count in counts.items()}

import math 
def calculate_scores(pair_counts, freqs, scale = 3):
    scores = {}
    total_pairs = sum(pair_counts.values())
    for pair, count in pair_counts.items():
        observed_freq = count / total_pairs
        expected_freq = freqs[pair[0]] * freqs[pair[1]]
        score = scale * math.log2(observed_freq / expected_freq)
        scores[pair] = round(score)
    return scores

nucleotides = ['A', 'G', 'C', 'T']
def create_blosum_matrix(scores, nucleotides):
    matrix = {}
    for i in nucleotides:
        matrix[i] = {}
        for j in nucleotides:
            if (i, j) in scores:
                matrix[i][j] = scores[(i, j)]
            elif (j, i) in scores:
                matrix[i][j] = scores[(j, i)]
    return matrix

def print_blosum_matrix(matrix, nucleotides):
    print("   " + " ".join(nucleotides))
    for i in nucleotides:
        row = [str(matrix[i][j]) for j in nucleotides]
        print(f"{i} " + " ".join(row))

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
def visualize_blosum_matrix(matrix, nucleotides):
    data = np.array([[matrix[x][y] for y in nucleotides] for x in nucleotides])
    plt.figure(figsize=(8, 6))
    sns.heatmap(data, annot=True, cmap="coolwarm", 
                xticklabels=nucleotides, yticklabels=nucleotides,
                fmt=".0f", cbar_kws={'label': 'BLOSUM Score'})
    plt.title("Матрица BLOSUM")
    plt.show()

def init(rows, cols, gap_penalty=10):
    matrix = [[0 for _ in range(cols + 1)] for _ in range(rows + 1)]
    for i in range(1, rows + 1):
        matrix[i][0] = -i * gap_penalty
    for j in range(1, cols + 1):
        matrix[0][j] = -j * gap_penalty
    return matrix

def get_new_score(up, left, middle, s_score, gap_penalty):
    return max(up - gap_penalty, left - gap_penalty, middle + s_score)

def align(top_seq, bottom_seq, gap_penalty, blosum_matrix):
    rows = len(top_seq)
    cols = len(bottom_seq)
    matrix = init(rows, cols, gap_penalty)
    for i in range(1, rows + 1):
        for j in range(1, cols + 1):
            s_score = blosum_matrix[top_seq[i - 1]][bottom_seq[j - 1]]
            matrix[i][j] = get_new_score(matrix[i - 1][j], 
                                         matrix[i][j - 1], 
                                         matrix[i - 1][j - 1], 
                                         s_score, 
                                         gap_penalty)
    return matrix

def get_alignment(top_seq, bottom_seq, sm, gap_penalty, blosum_matrix):
    top_aligned = ""
    bottom_aligned = ""
    i = len(top_seq)
    j = len(bottom_seq)
    while i > 0 or j > 0:
        if (i > 0 and j > 0 and 
            sm[i][j] == sm[i - 1][j - 1] + blosum_matrix[top_seq[i - 1]][bottom_seq[j - 1]]):
            top_aligned = top_seq[i - 1] + top_aligned
            bottom_aligned = bottom_seq[j - 1] + bottom_aligned
            i -= 1
            j -= 1
        elif i > 0 and sm[i][j] == sm[i - 1][j] - gap_penalty:
            top_aligned = top_seq[i - 1] + top_aligned
            bottom_aligned = "-" + bottom_aligned
            i -= 1
        else:
            top_aligned = "-" + top_aligned
            bottom_aligned = bottom_seq[j - 1] + bottom_aligned
            j -= 1
    return top_aligned, bottom_aligned

top_seq = "AGTACGCA"
bottom_seq = "TATGC"
gap_penalty = 2
sm = align(top_seq, bottom_seq, gap_penalty, blosum_matrix)
aligns = get_alignment(top_seq, bottom_seq, sm, gap_penalty, blosum_matrix)
print(aligns[0])
print(aligns[1])

print()

top_seq = "AGTCTCCCCC"
bottom_seq = "ACTTCTACCCCAGC"
sm = align(top_seq, bottom_seq, gap_penalty, blosum_matrix)
aligns = get_alignment(top_seq, bottom_seq, sm, gap_penalty, blosum_matrix)
print(aligns[0])
print(aligns[1])

AGTACGCA
--TATGC-

AG-TCT-CCCC--C
ACTTCTACCCCAGC
